##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: File API Quickstart

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/File_API.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

The Gemini API supports prompting with text, image, and audio data, also known as *multimodal* prompting. You can include text, image,
and audio in your prompts. For small images, you can point the Gemini model
directly to a local file when providing a prompt. For larger text files, images, videos, and audio, upload the files with the [File
API](https://ai.google.dev/api/rest/v1beta/files) before including them in
prompts.

The File API lets you store up to 20GB of files per project, with each file not
exceeding 2GB in size. Files are stored for 48 hours and can be accessed with
your API key for generation within that time period. It is available at no cost in all regions where the [Gemini API is
available](https://ai.google.dev/available_regions).

For information on valid file formats (MIME types) and supported models, see the documentation on
[supported file formats](https://ai.google.dev/tutorials/prompting_with_media#supported_file_formats)
and view the text examples at the end of this guide.

This guide shows how to use the File API to upload a media file and include it in a `GenerateContent` call to the Gemini API. For more information, see the [code
samples](https://github.com/google-gemini/cookbook/tree/main/quickstarts/file-api).


In [1]:
!pip install -U -q google-generativeai

In [2]:
import google.generativeai as genai


## Authentication

**Important:** The File API uses API keys for authentication and access. Uploaded files are associated with the API key's cloud project. Unlike other Gemini APIs that use API keys, your API key also grants access data you've uploaded to the File API, so take extra care in keeping your API key secure. For best practices on securing API keys, refer to Google's [documentation](https://support.google.com/googleapi/answer/6310037).

### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [3]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

## Upload file

The File API lets you upload a variety of multimodal MIME types, including images and audio formats. The File API handles inputs that can be used to generate content with [`model.generateContent`](https://ai.google.dev/api/rest/v1/models/generateContent) or [`model.streamGenerateContent`](https://ai.google.dev/api/rest/v1/models/streamGenerateContent).

The File API accepts files under 2GB in size and can store up to 20GB of files per project. Files last for 2 days and cannot be downloaded from the API.

First, you will prepare a sample image to upload to the API.

Note: You can also [upload your own files](https://github.com/google-gemini/cookbook/tree/main/examples/Upload_files_to_Colab.ipynb) to use.

## Supported text types

As well as supporting media uploads, the File API can be used to embed text files, such as Python code, or Markdown files, into your prompts.

This example shows you how to load a markdown file into a prompt using the File API.

In [4]:
# Download a markdown file and ask a question.

!curl -so contrib.md https://raw.githubusercontent.com/google-gemini/cookbook/main/CONTRIBUTING.md



In [5]:
md_file = genai.upload_file(path="contrib.md", display_name="Contributors guide", mime_type="text/markdown")

model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")
response = model.generate_content(
    [
        "What should I do before I start writing, when following these guidelines?",
        md_file,
    ]
)
print(response.text)

Here's a breakdown of what you need to do before writing, based on the provided guidelines:

**Essentials:**

1. **Sign the Google CLA:**
   -  This legal agreement is mandatory for all contributions.
   - Check if you or your employer have already signed it: [https://cla.developers.google.com/](https://cla.developers.google.com/)
   - If not, sign a new one through the link above.

2. **Familiarize Yourself with Style Guides:**
   -  **Technical Writing:** Skim the highlights for common feedback points: [https://developers.google.com/style/highlights](https://developers.google.com/style/highlights)
   -  **Language-Specific:**  Identify the style guide for your chosen language (e.g., Python, JavaScript) here: [https://google.github.io/styleguide/](https://google.github.io/styleguide/)
   - **Python Notebooks:** If applicable, consider using `pyink` for style consistency in your `.ipynb` files.

3. **Content Submission Protocol:**
   - **File an Issue:** *Before* writing any code, subm

Some common text formats are automatically detected, such as `text/x-python`, `text/html` and `text/markdown`. If you are using a file that you know is text, but is not automatically detected by the API as such, you can specify the MIME type as `text/plain` explicitly.

In [6]:
# Download some C++ code and force the MIME as text when uploading.

!curl -so gemma.cpp https://raw.githubusercontent.com/google/gemma.cpp/main/examples/hello_world/run.cc



In [7]:
cpp_file = genai.upload_file(
    path="gemma.cpp", display_name="gemma.cpp", mime_type="text/plain"
)



In [8]:
model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")
response = model.generate_content(["What does this program do?", cpp_file])
print(response.text)

This C++ program demonstrates the use of the **Gemma** language model for text generation. Let's break down what the code does step-by-step:

**1. Initialization and Setup**

   * **Includes:** The code includes necessary header files for input/output, the Gemma library (`gemma.h`), thread pool management, and command-line argument parsing.
   * **Licensing:** The initial lines specify the licensing (Apache 2.0) for the code.
   * **Thread Pool:** A thread pool (`hwy::ThreadPool`) is created to leverage multi-threading for potentially faster inference.
   * **Model and Cache Loading:**
      * `gcpp::LoaderArgs loader(argc, argv);`  parses command-line arguments likely related to model loading (e.g., paths to weight files).
      *  `gcpp::Gemma model(loader.tokenizer, loader.weights, loader.ModelType(), pool);`  instantiates the Gemma model, using the specified tokenizer, model weights, model type, and the thread pool.
      * `auto kv_cache = CreateKVCache(loader.ModelType());` creat